In [1]:
# Imports 
import DataFrameInfo_class as info
import pandas as pd 
from scipy.stats import normaltest 
from statsmodels.graphics.gofplots import qqplot 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load data 
failure_data = pd.read_csv('failure_data_after_data_transformation.csv')

In [3]:
# Plotter Class

class Plotter: 
    def __init__(self, df):
        self.df = df 
    
    def plot_hist(self, column_name):
        self.df[column_name].hist(bins = 40)
        plt.show()

    def normal_test(self, column_name):
        stat, p = normaltest(self.df[column_name], nan_policy = 'omit')
        print('Statistics=%.3f, p=%.3f' % (stat, p))
    
    def plot_qq(self, column_name):
        qq_plot_air_temp = qqplot(self.df[column_name], scale=1, line ='q')
        plt.show()

    def print_mean(self, column_name):
        print(f'The mean of {column_name} is {self.df[column_name].mean()}')
    
    def print_median(self, column_name):
        print(f'The median of {column_name} is {self.df[column_name].median()}')

In [ ]:
# add analysis from plotter_v2.py
# The majority of this is for skewness 

# brain dump 
# use the missing data analysis and put it here from the longer script 
# perform treatement for any skewed data 
# data transformations
# update the DataTransformation Class 
# data cleaning:
    # data info 
    # MCAR / MAR / NMAR? - I have answered this a little.

In [4]:
# The following Data Transformation act as a treatement to missing data in the data set 

# Treating the NULL values 
failure_data['Air temperature [K]'] = failure_data['Air temperature [K]'].fillna(failure_data['Air temperature [K]'].median())
failure_data['Process temperature [K]'] = failure_data['Process temperature [K]'].fillna(failure_data['Process temperature [K]'].median())
failure_data.dropna(subset='Tool wear [min]', inplace=True)

In [5]:
# create a new variable after treating the null variables
failure_data_without_null = failure_data

In [ ]:
# Analysis & visualisatoin ideas 

In [ ]:
# EDA ideas 

# check the balance of the data set for `machine failure` variable.
    # do this is with a bar chart 
# Do a correlation matrix of all numerical variables, where the dependant (y) variable is 'machine failure' 

# categorical plots 
# correcting the skew - do this first for the independant variables that have the highest correlation 
# count plots of our nominal category data (those with binary data)
# summary plots - pairplot 

In [6]:
failure_data_without_null['Machine failure'].value_counts()
# Data set is heavyly in balanced, this will cause a bias in ML 

Machine failure
0    9188
1     328
Name: count, dtype: int64

In [ ]:

    # do this is with a bar chart 
# Do a correlation matrix of all numerical variables, where the dependant (y) variable is 'machine failure' 

# categorical plots 
# correcting the skew - do this first for the independant variables that have the highest correlation 
# count plots of our nominal category data (those with binary data)
# summary plots - pairplot 

# create a bar plot of purpose vs amount
sns.barplot(data=failure_data_after_transformations['Machine failure'])